# Training Set

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.externals import joblib
%matplotlib inline   

In [2]:
import os 
os.system('ps aux | grep wolfm2')
#os.system('killall -s SIGKILL -u wolfm2')
#os.system('cp /home/wolfm2/job.sh .; echo test 1>&2') #; cp ../job.log ../jerbb.txt')

isTraining = False

### Read raw training data

In [3]:
if isTraining:
    amazon = pd.read_csv('/home/wolfm2/amazon_data.0/raw_data_train.csv')
    #amazon = pd.read_csv('/home/eydu/amazon_data/raw_data_train.csv')
    #amazon = pd.read_csv('/home/ich/amazon_data/raw_data_train.csv')
else:
    amazon = pd.read_csv('/home/wolfm2/amazon_data.0/raw_data_test.csv')

print(amazon.shape)

(136500, 14)


In [4]:
print(amazon.head())
print(amazon['helpful'].mean())

   Unnamed: 0  Unnamed: 0.1      Id   ProductId          UserId  \
0      264540        182243  182244  B005OSC218   AJZXTZ3I9A84H   
1      146085        548350  548351  B004VLVD50  A39RNG8DUXG7ZM   
2      281714        539422  539423  B001BDDT8K  A2FRFAQCWZJT3Q   
3      280559        119141  119142  B001U0ON5M  A1YREIUL7VDQ0B   
4      236948        352256  352257  B000E63L8S  A3JKXNOLX2QRJ7   

                         ProfileName  HelpfulnessNumerator  \
0                             Jeromy                     1   
1                         W. Chapman                     4   
2        B. Davis "The Happy Hermit"                     1   
3   Jennifer U. Heston "ex-academic"                     0   
4  darlene conright "alaskan darlin"                     2   

   HelpfulnessDenominator  Score        Time                  Summary  \
0                       1      5  1316995200                 The Best   
1                       4      5  1331596800  Makes great dog bones !   
2    

### Feature extraction on natural language data

In [5]:
# # http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# corpus = amazon.Text.as_matrix()
# X_bag_of_words = vectorizer.fit_transform(corpus)
# print(X_bag_of_words.toarray())

In [6]:
import nltk
# nltk.download('punkt')
# nltk.download('popular')

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer

# amazon['reviewLen'] = amazon['Text'].str.len() # Use this later /X as gross dummy

amazon['textLower'] = amazon['Text'].str.lower()
# look at the text strip_accents=ascii,  stop_words={'english'}, token_pattern = r'\b[a-zA-Z0-9]{3,}\b'),
if isTraining:
    hv0 = HashingVectorizer(n_features=2 ** 19, non_negative=True, tokenizer=LemmaTokenizer(), ngram_range=(1,3))
    X_hv0 = hv0.fit_transform(amazon.textLower)
else:
    hv0 = joblib.load('hv0.pkl') # pickle
    X_hv0 = hv0.transform(amazon.textLower)


amazon['summaryFilter'] = amazon['Summary'].apply(lambda x: " " if x is np.nan else x) # some were np.nans
amazon['sfLower'] = amazon['summaryFilter'].str.lower()
# # and a second domain where we look at the summary
if isTraining:
    hv1 = HashingVectorizer(n_features=2 ** 19, non_negative=True, tokenizer=LemmaTokenizer(), ngram_range=(1,3))
    X_hv1 = hv1.fit_transform(amazon.sfLower) 
else:
    hv1 = joblib.load('hv1.pkl') # pickle
    X_hv1 = hv1.transform(amazon.sfLower) 

# Another hash domain we want to count but not scale
# amazon['timeFilter'] = amazon['Time'].apply(lambda x: str(int(x)%(86400 * 7))) # converts to day of week
# hv2 = HashingVectorizer(n_features=2 ** 17, non_negative=True, strip_accents=ascii, 
#                            ngram_range=(1,1)) 
# X_hv2 = hv2.fit_transform(amazon.timeFilter + " " + amazon.ProductId + " " + amazon.UserId) # mw adds uid as token

amazon['logReviewLen'] = np.round(np.log(amazon['Text'].str.len()),decimals=1) + 10
amazon.hist(column="logReviewLen")

amazon['ScoreX'] = amazon['Score'].apply(lambda x: str(x)) # make score acceptable
amazon['sLogReviewLen'] = amazon['logReviewLen'].apply(lambda x: str(x)) # make score acceptable
if isTraining:
    hv2 = HashingVectorizer(n_features=2 ** 17, non_negative=True, ngram_range=(1,1)) 
    X_hv2 = hv2.fit_transform(amazon.ScoreX + " " + amazon.UserId + " " + amazon.sLogReviewLen) # mw adds uid as token
else:
    hv2 = joblib.load('hv2.pkl') # pickle
    X_hv2 = hv2.transform(amazon.ScoreX + " " + amazon.UserId + " " + amazon.sLogReviewLen) # mw adds uid as token

import scipy.sparse as sp
X_hv = sp.hstack([X_hv0, X_hv1], format='csr')
print(X_hv.shape)

In [ ]:
# x = amazon.UserId + " " +  amazon.Text
# x.head(10)

In [ ]:
# We want to be able to use this model fit on other data (the test set)
# So let's save a copy of this instance of HashingVectorizer to be able to transform other data with this fit
# http://scikit-learn.org/stable/modules/model_persistence.html
if isTraining:
    joblib.dump(hv0, 'hv0.pkl') # pickle
    joblib.dump(hv1, 'hv1.pkl') # pickle
    joblib.dump(hv2, 'hv2.pkl') # pickle

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
from sklearn.feature_extraction.text import TfidfTransformer
if isTraining:
    transformer = TfidfTransformer()
    X_tfidf = transformer.fit_transform(X_hv)
    joblib.dump(transformer, 'transformer.pkl') # pickle
else:
    transformer = joblib.load('transformer.pkl') # pickle
    X_tfidf = transformer.transform(X_hv)
    

In [ ]:
print(type(X_tfidf))

### Create additional quantitative features

In [ ]:
# features from Amazon.csv to add to feature set
import re

#amazon['reviewLen'] = amazon['Text'].str.len()
#amazon['summaryLen'] = amazon['summaryFilter'].str.len()

#amazon['rlMeanDist'] = amazon['reviewLen'].apply(lambda x: abs(x-80)) # 80 is avg summary len. Thx George!
#amazon['slMeanDist'] = amazon['summaryLen'].apply(lambda x: abs(x-8)) # 8. just guessing here.

#import zlib
#amazon['nameHash'] = zlib.crc32(str(amazon['UserId']).encode('utf8'))
#amazon['nameHash'] = amazon['UserId'].apply(lambda x: zlib.crc32(str(x).encode('utf8'))) # bad. don't do it this way

# stackoverflow.com/questions/15772371/finding-average-length-of-items-in-a-list-python
# averages array element lengths
def avgLen(text, regex):
    lst = re.findall(regex, text)
    lengths = [len(i) for i in lst]
    return 0 if len(lengths) == 0 else (float(sum(lengths)) / len(lengths)) 

# ratio of regex to whole
def cRatio(text, regex):
    num = len(re.findall(regex, text))
    text = "" if text is np.nan else text
    den = len(text)
    return 0 if den == 0 else num / den

# Review Len
amazon['summaryLen'] = amazon['Text'].str.len()

# Num Words
amazon['numWords'] = amazon['Text'].apply(lambda x: len(re.findall("[a-zA-Z']+", x)))

# Num Cap Words
amazon['numCapWords'] = amazon['Text'].apply(lambda x: len(re.findall("[A-Z']+", x)))

# Avg Sentence Len
amazon['avgSenLen'] = amazon['Text'].apply(lambda x: avgLen(x, "[a-zA-Z' ]+"))

# Avg Word Len
amazon['avgWrdLen'] = amazon['Text'].apply(lambda x: avgLen(x, "[a-zA-Z']+"))

# ! Ratio
amazon['ratioBang'] = amazon['Text'].apply(lambda x: cRatio(x, "\!"))

# ? Ratio                             
amazon['ratioQmark'] = amazon['Text'].apply(lambda x: cRatio(x, "\?"))

# X_quant_features = amazon[["Score", "reviewLen", "summaryLen", "rlMeanDist", "slMeanDist"]]
# print(X_quant_features.head(10))
# print(type(X_quant_features))
X_quant_features = amazon[['summaryLen', 'numWords', 'numCapWords', 'avgSenLen', 'avgWrdLen', 'ratioBang', 'ratioQmark']]

### Combine all quantitative features into a single sparse matrix

In [ ]:
from scipy.sparse import csr_matrix, hstack
X_quant_features_csr = csr_matrix(X_quant_features)
X_combined = hstack([X_tfidf, X_quant_features_csr, X_hv2])  # we dont want to penalize hv2 w tfidf MW
X_matrix = csr_matrix(X_combined) # convert to sparse matrix
print(X_matrix.shape)

### Create `X`, scaled matrix of features

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
if isTraining:
    sc = StandardScaler(with_mean=False)
    X = sc.fit_transform(X_matrix)
    joblib.dump(sc, 'sc.pkl') # pickle
else:
    sc = joblib.load('sc.pkl')
    X = sc.transform(X_matrix)

print(X.shape)

### create `y`, vector of Labels

In [ ]:
y = amazon['helpful'].values
print(type(y))

### fit models

In [ ]:
from my_measures import BinaryClassificationPerformance

In [ ]:
# # MODEL: SVM, linear
# from sklearn import linear_model
# svm = linear_model.SGDClassifier()
# svm.fit(X, y)

# joblib.dump(svm, 'svm.pkl') # pickle

if not isTraining:
    svm = joblib.load('best.svm.pkl')
    svm_performance = BinaryClassificationPerformance(svm.predict(X), y, 'svm')
    svm_performance.compute_measures()
    print(svm_performance.performance_measures)

In [ ]:
# # MODEL: logistic regression
# from sklearn import linear_model
# #lgs = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
# lgs = linear_model.SGDClassifier(loss='log', n_iter=1000, alpha=0.1)

# lgs.fit(X, y)
# joblib.dump(lgs, 'lgs.pkl') # pickle
if not isTraining:
    lgs = joblib.load('best.lgs.pkl')
    lgs_performance = BinaryClassificationPerformance(lgs.predict(X), y, 'lgs')
    lgs_performance.compute_measures()
    print(lgs_performance.performance_measures)

In [ ]:
# # MODEL: Naive Bayes
# from sklearn.naive_bayes import MultinomialNB
# nbs = MultinomialNB()
# nbs.fit(X, y)
# joblib.dump(nbs, 'nbs.pkl') # pickle
if not isTraining:
    nbs = joblib.load('best.nbs.pkl')
    nbs_performance = BinaryClassificationPerformance(nbs.predict(X), y, 'nbs')
    nbs_performance.compute_measures()
    print(nbs_performance.performance_measures)

In [ ]:
# # MODEL: Ridge Regression Classifier
# from sklearn import linear_model
# rdg = linear_model.RidgeClassifier()
# rdg.fit(X, y)
# joblib.dump(rdg, 'rdg.pkl') # pickle
if not isTraining:
    rdg = joblib.load('best.rdg.pkl')
    rdg_performance = BinaryClassificationPerformance(rdg.predict(X), y, 'rdg')
    rdg_performance.compute_measures()
    print(rdg_performance.performance_measures)

In [ ]:
# # MODEL: Perceptron
# from sklearn import linear_model
# prc = linear_model.SGDClassifier(loss='perceptron')
# prc.fit(X, y)
# joblib.dump(prc, 'prc.pkl') # pickle
if not isTraining:
    prc = joblib.load('best.prc.pkl')
    prc_performance = BinaryClassificationPerformance(prc.predict(X), y, 'prc')
    prc_performance.compute_measures()
    print(prc_performance.performance_measures)

In [ ]:
import json
import datetime
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

from sklearn.neural_network import MLPClassifier # mw

# prepare a range of alpha values to test
# alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
alphas = np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001])
Cs = np.array([0.001, 0.01, 0.1, 1, 10, 100, 1000])
# create and fit a ridge regression model, testing each alpha
# model = linear_model.SGDClassifier(loss='perceptron', max_iter=50) # max_iter 1000

if isTraining:
    mlp = MLPClassifier(random_state=0)
    svm = linear_model.SGDClassifier(n_iter=500)
    lgs = linear_model.SGDClassifier(loss='log', n_iter=500)
    nbs = MultinomialNB()
    rdg = linear_model.RidgeClassifier()
    prc = linear_model.SGDClassifier(loss='perceptron', n_iter=500)
    mList = [[svm,"svm"], [lgs,"lgs"], [prc,"prc"], [nbs,"nbs"], [rdg,"rdg"]]
else:
    mList = []

for model in mList: 
# for model in []: 
# for model in [rdg]:    
  fh = open("GridSearch.txt", "a")
  grid = GridSearchCV(estimator=model[0], param_grid=dict(alpha=alphas), n_jobs=2) #
  grid.fit(X, y)
  print(grid)
  # summarize the results of the grid search
  print(grid.cv_results_)
  print(grid.best_score_)
  print(grid.best_estimator_.alpha)

  fh.write('\n########\n')
  fh.write(str(datetime.datetime.now()))
  fh.write('\n########\n')
  fh.write(str(model[0]) + '\n')  
  fh.write(str(grid.cv_results_).replace(", '", ",\n'") + '\n')
  fh.write(str(grid.best_score_) + '\n')  
  fh.write(str(grid.best_estimator_.alpha) + '\n')
  fh.close()

  # MODEL: BEST
  best = grid.best_estimator_

  best.fit(X, y)
  joblib.dump(best, 'best.{}.pkl'.format(model[1])) # pickle

  best_performance = BinaryClassificationPerformance(best.predict(X), y, 'best')
  best_performance.compute_measures()
  print(best_performance.performance_measures)
  

In [ ]:
'''
pg = {'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,1), (100,2), (100,3)],
#'alpha': [10.0 ** -np.arange(1, 7)],
'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001],
'activation': ["logistic", "relu", "Tanh"],
'tol': [1e-2, 1e-4, 1e-6],
'epsilon': [1e-3, 1e-7, 1e-8, 1e-9, 1e-8]
}

fh = open("GridSearch.txt", "a")
grid = GridSearchCV(estimator=mlp, param_grid=pg, n_jobs=2) #
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.cv_results_)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

fh.write('\n########\n')
fh.write(str(datetime.datetime.now()))
fh.write('\n########\n')
fh.write(str(model) + '\n')  
fh.write(str(grid.cv_results_).replace(", '", ",\n'") + '\n')
fh.write(str(grid.best_score_) + '\n')  
fh.write(str(grid.best_estimator_.alpha) + '\n')
fh.close()
'''

In [ ]:
'''
# MODEL: BEST
best = grid.best_estimator_

best.fit(X, y)
joblib.dump(best, 'best.pkl') # pickle

best_performance = BinaryClassificationPerformance(best.predict(X), y, 'best')
best_performance.compute_measures()
print(best_performance.performance_measures)
'''

### ROC plot to compare performance of various models and fits

In [ ]:
fits = [svm_performance, lgs_performance, nbs_performance, rdg_performance, prc_performance]
# fits = [svm_performance, lgs_performance, rdg_performance, prc_performance]

if isTraining:
    fList = []
else:
    fList = fits

for fit in fList:
    plt.plot(fit.performance_measures['FP'] / fit.performance_measures['Neg'], 
             fit.performance_measures['TP'] / fit.performance_measures['Pos'], 'ro')
    plt.text(fit.performance_measures['FP'] / fit.performance_measures['Neg'], 
             fit.performance_measures['TP'] / fit.performance_measures['Pos'], fit.desc)
plt.axis([0, 1, 0, 1])
plt.title('ROC plot: training set')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()